##  Exercise II: Levels of Detail

*Carolina Giménez Arias*

In [1]:
import numpy 
from stl import mesh
import vtk
import time

In [6]:
# Cargamos el archivo que contiene la aorta
aorta = vtk.vtkSTLReader()
aorta.SetFileName("a1.stl")

In [7]:
# Iniciamos ventana
rw = vtk.vtkRenderWindow()
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(rw)

# Rangos del viewport
xmins=[0,0.5,0,0.5]
xmaxs=[0.5,1,0.5,1]
ymins=[0,0,0.5,0.5]
ymaxs=[0.5,0.5,1,1]

dreduc = [0, 0.3, 0.6, 0.9] #niveles de reduccion del número de triángulos (decimate)
slevels = [1,500,3000,6000] #niveles de smooth

for i in range(4):
    # Creamos renderer
    ren = vtk.vtkRenderer()
    rw.AddRenderer(ren)        
    ren.SetViewport(xmins[i],ymins[i],xmaxs[i],ymaxs[i])
    
    # Decimate
    decimate = vtk.vtkDecimatePro()
    decimate.SetInputConnection(aorta.GetOutputPort())
    decimate.SetTargetReduction(dreduc[i])
    
    # Smooth
    smoother = vtk.vtkSmoothPolyDataFilter()
    smoother.SetInputConnection(decimate.GetOutputPort())
    smoother.SetNumberOfIterations(slevels[i])
    
    # Creamos mapper y actor
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(smoother.GetOutputPort())     
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(0.9,0.4,0.2) # (R,G,B) Le damos el color naranja
    
    ren.AddActor(actor)
    ren.ResetCamera()
    
rw.Render()
rw.SetWindowName('Multiple levels of detail of the aorta')
iren.Start()


Se observa como a medida que reducimos el número de triángulos con el decimate, la figura tiene una forma más "ruda", lo que provoca que sea más complicado suavizarla con el smooth (razón por la cuál la aorta de la esquina superior derecha cobra esa forma). Cabe resaltar que el orden  de las imágenes sale invertido (3,4,1,2)